In [60]:
import torch.nn as nn
import torch.nn.functional as F

device = 'cpu'
batch_size = 32

temp = 0.7
N = 2 * batch_size

z_i, z_j = F.normalize(torch.randn(batch_size, 100)), F.normalize(torch.randn(batch_size, 100))
z = torch.cat((z_i, z_j), dim=0)

sim = torch.exp(torch.mm(z, z.t()) / temp)

mask = ~torch.eye(N, device=device).bool()
negatives = sim.masked_select(mask).view(N, -1)
negatives = torch.sum(negatives, dim=-1)

positives = torch.exp(torch.sum(z_i * z_j, dim=-1) / temp)
positives = torch.cat([positives, positives])

loss = -torch.log(positives / negatives)
loss = torch.mean(loss)


tensor(4.1850)
